In [ ]:
import os
import aisdb
import pickle
import numpy as np
import pandas as pd
import nest_asyncio
import geopandas as gpd
import cartopy.crs as ccrs
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cartopy.feature as cfeature
from aisdb import DBConn, DBQuery, DomainFromPoints
from aisdb.database.dbconn import PostgresDBConn
from shapely.geometry import Point, MultiPoint
from aisdb.webdata.shore_dist import CoastDist
from aisdb.webdata.shore_dist import ShoreDist
from aisdb.webdata.shore_dist import PortDist
from datetime import datetime, timedelta
from aisdb.gis import DomainFromTxts
from aisdb.database import sqlfcn_callbacks
from shapely import prepare
from PIL import ImageFile
from tqdm import tqdm
nest_asyncio.apply()
ImageFile.LOAD_TRUNCATED_IMAGES = True
from aisdb.weather.era5 import ClimateDataStore # for weather


In [ ]:
# >>> PostgreSQL Information <<<
db_user='<user-name>'            # DB User
db_dbname='aisviz'         # DB Schema
db_password='<password>'    # DB Password
db_hostaddr='127.0.0.1'    # DB Host address

dbconn = PostgresDBConn(
    port=5555,             # PostgreSQL port
    user=db_user,          # PostgreSQL username
    dbname=db_dbname,      # PostgreSQL database
    host=db_hostaddr,      # PostgreSQL address
    password=db_password,  # PostgreSQL password
)

In [ ]:
xmin, ymin, xmax, ymax = -70, 45, -58, 53
gulf_bbox = [xmin, xmax, ymin, ymax]
start_time = datetime(2023, 8, 1)
end_time = datetime(2023, 8, 30)

qry = DBQuery(
    dbconn=dbconn,
    start=start_time, end=end_time,
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax,
    callback=aisdb.database.sqlfcn_callbacks.in_time_bbox_validmmsi
)

ais_tracks = []
rowgen = qry.gen_qry()
tracks = aisdb.track_gen.TrackGen(rowgen, decimate=True")

In [ ]:
climate_data_store = ClimateDataStore(['10v','10u'], start_time, end_time,"/home/CanadaV2")

In [ ]:
import csv

# Define the headers for the CSV file
headers = ['mmsi', 'time', 'lon', 'lat', 'cog', 'sog',
           'utc_second', 'heading', 'rot', 'maneuver','10v','10u','u10','v10']

# Open the CSV file for writing
csv_filename = '2023-08.csv'

# Assuming `climate_data_store.extract_weather()` returns a dictionary
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()  # Write the header once

    for track in tracks:
        for i in range(len(track['time'])):
            # Extract weather data for the current location and time
            values = climate_data_store.extract_weather(track['lat'][i], track['lon'][i], track['time'][i])

            # Flatten the row
            row = {
                'rot': track['rot'],
                'mmsi': track['mmsi'],
                'lon': track['lon'][i],
                'lat': track['lat'][i],
                'cog': track['cog'][i],
                'sog': track['sog'][i],
                'time': track['time'][i],
                'heading': track['heading'],
                'maneuver': track['maneuver'],
                'utc_second': track['utc_second'][i],
            }

            # Add weather data (values) as additional fields in the row
            for key, value in values.items():
                row[key] = value

            # Write the row to the CSV file
            writer.writerow(row)


print(f"All tracks have been combined and written to {csv_filename}")